In [25]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import matplotlib.pyplot as plt
from scipy.stats import pearsonr
import torch
from torch.utils.data import TensorDataset, DataLoader
from sklearn.metrics import mean_squared_error, r2_score

In [26]:
weather = pd.read_csv("../Weather/Weather.csv")
weather = weather.drop(columns=["%time"])
resource = pd.read_csv("../AICU/Resources.csv")
test_resource = pd.read_csv("../TheAutomators/Resources.csv")
climate = pd.read_csv("../AICU/GreenhouseClimate.csv",encoding='utf-8')
test_climate = pd.read_csv("../TheAutomators/GreenhouseClimate.csv",encoding='utf-8')
production = pd.read_csv("../TheAutomators/Production.csv",encoding='utf-8')
production = production.astype(float)
climate=climate.astype(float)

C:\Users\thanh\AppData\Local\Temp\ipykernel_3536\3084357586.py:5: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,14,15,17,18,20,22,23,25,27,29,31,33,35,37,39,41,43,44,45,47,49) have mixed types. Specify dtype option on import or set low_memory=False.
  climate = pd.read_csv("../AICU/GreenhouseClimate.csv",encoding='utf-8')
C:\Users\thanh\AppData\Local\Temp\ipykernel_3536\3084357586.py:6: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,14,15,17,18,20,22,23,25,27,29,31,33,35,37,39,41,43,44,45,47,49) have mixed types. Specify dtype option on import or set low_memory=False.
  test_climate = pd.read_csv("../TheAutomators/GreenhouseClimate.csv",encoding='utf-8')


In [27]:
sample_per_day = 288 # 5 phút 
N_days = len(weather) // sample_per_day

# Reshape input: [samples, timesteps, features]
X_seq = []
for i in range(N_days):
    start = i * sample_per_day
    end = start + sample_per_day
    X_seq.append(weather.iloc[start:end].values)

X_seq = np.array(X_seq)  # Shape: (166, 288, N_weather)
y = resource["Heat_cons"].values.reshape(-1,1)   # Shape: (166, N_resource)

# Standarize
scaler_X = StandardScaler()
X_seq_scaled = np.array([scaler_X.fit_transform(day) for day in X_seq])
X_seq_scaled = np.nan_to_num(X_seq_scaled)

scaler_y = StandardScaler()
y_scaled = scaler_y.fit_transform(y)
y_scaled = np.nan_to_num(y_scaled)
# Split train/test
X_train, X_test, y_train, y_test = train_test_split(X_seq_scaled, y_scaled, test_size=0.2, random_state=42)
print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")


X_train shape: (132, 288, 10), y_train shape: (132, 1)


In [28]:
# Assuming X_train and y_train are numpy arrays
X_tensor = torch.tensor(X_train, dtype=torch.float32)  # shape: (132, 288, 10)
y_tensor = torch.tensor(y_train, dtype=torch.float32)  # shape: (132, 1)

dataset = TensorDataset(X_tensor, y_tensor)
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)

In [29]:
import torch.nn as nn

class GRUModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(GRUModel, self).__init__()
        self.gru = nn.GRU(
            input_size=input_size,   # 10
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True         # input: (batch, seq, feature)
        )
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        # x shape: (batch, seq_len, input_size)
        out, _ = self.gru(x)         # out shape: (batch, seq_len, hidden_size)
        out = out[:, -1, :]          # take last timestep: (batch, hidden_size)
        out = self.fc(out)           # shape: (batch, output_size)
        return out


In [57]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = GRUModel(input_size=10, hidden_size=64, num_layers=2, output_size=1).to(device)
criterion = nn.MSELoss()  # or BCEWithLogitsLoss() if binary classification
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

num_epochs = 30
for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    all_preds = []
    all_targets = []

    for batch_X, batch_y in dataloader:
        batch_X = batch_X.to(device)
        batch_y = batch_y.to(device)

        optimizer.zero_grad()
        outputs = model(batch_X)

        loss = criterion(outputs, batch_y)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()

        total_loss += loss.item()

        # Store predictions and targets
        all_preds.append(outputs.detach().cpu())
        all_targets.append(batch_y.detach().cpu())

    # Compute R² after entire epoch
    all_preds = torch.cat(all_preds).numpy()
    all_targets = torch.cat(all_targets).numpy()
    r2 = r2_score(all_targets, all_preds)

    print(f"Epoch [{epoch+1}/{num_epochs}] Loss: {total_loss:.4f} | R²: {r2:.4f}")



Epoch [1/30] Loss: 9.2296 | R²: -0.0062
Epoch [2/30] Loss: 8.8684 | R²: 0.0485
Epoch [3/30] Loss: 8.2435 | R²: 0.0732
Epoch [4/30] Loss: 9.0205 | R²: 0.0967
Epoch [5/30] Loss: 7.7670 | R²: 0.1256
Epoch [6/30] Loss: 7.4033 | R²: 0.1645
Epoch [7/30] Loss: 7.0824 | R²: 0.2156
Epoch [8/30] Loss: 6.5223 | R²: 0.2687
Epoch [9/30] Loss: 6.2591 | R²: 0.3197
Epoch [10/30] Loss: 6.2717 | R²: 0.3261
Epoch [11/30] Loss: 6.9525 | R²: 0.3265
Epoch [12/30] Loss: 5.6703 | R²: 0.3627
Epoch [13/30] Loss: 5.7849 | R²: 0.3612
Epoch [14/30] Loss: 5.6614 | R²: 0.3914
Epoch [15/30] Loss: 4.9963 | R²: 0.4329
Epoch [16/30] Loss: 4.9083 | R²: 0.4382
Epoch [17/30] Loss: 4.8324 | R²: 0.4439
Epoch [18/30] Loss: 4.8184 | R²: 0.4824
Epoch [19/30] Loss: 5.0736 | R²: 0.5056
Epoch [20/30] Loss: 4.2379 | R²: 0.5312
Epoch [21/30] Loss: 4.5542 | R²: 0.5355
Epoch [22/30] Loss: 4.0754 | R²: 0.5405
Epoch [23/30] Loss: 4.6314 | R²: 0.5172
Epoch [24/30] Loss: 3.6266 | R²: 0.5952
Epoch [25/30] Loss: 3.9645 | R²: 0.6141
Epoch [2

In [58]:
torch.save(model.state_dict(), 'gru_model.pth')


In [59]:
# Define your GRU model class (should be same as training)
class GRUNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(GRUNet, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.gru(x, h0)
        out = self.fc(out[:, -1, :])  # Use last timestep output
        return out

# Initialize model
model = GRUNet(input_size=10, hidden_size=64, num_layers=2, output_size=1)
model.load_state_dict(torch.load('gru_model.pth'))
model.eval()  # Set to evaluation mode


GRUNet(
  (gru): GRU(10, 64, num_layers=2, batch_first=True)
  (fc): Linear(in_features=64, out_features=1, bias=True)
)

In [60]:
import torch
from sklearn.metrics import r2_score

# Assume device, model, scaler_y, X_test, y_test are defined already

# Prepare X_test tensor and move to device
X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)
model.to(device)
# Put model in eval mode and predict
model.eval()
with torch.no_grad():
    y_pred_scaled = model(X_test_tensor).cpu().numpy()
# Inverse transform predictions and true targets back to original scale
y_pred = scaler_y.inverse_transform(y_pred_scaled)
y_test_original = scaler_y.inverse_transform(y_test)

# Calculate and print R² score
r2 = r2_score(y_test_original, y_pred)
print(f"Test R²: {r2:.4f}")


Test R²: -0.6760
